# Bottom up inventory generation

This script attempts to generate a bottom-up inventory as an example for the users.

We will generate the emissions for few sectors using various data sources.

We decided here to create the inventory for the
[Corvo island](https://en.wikipedia.org/wiki/Corvo_Island) in the Azores
as it is a small island so it will not cost a lot of processing and will be
quite fun to do.
It is a volcanic island with a population of around 384 people (2021).
The biggest activites are agriculture and tourism.

In [ ]:
import logging

import geopandas as gpd
import pandas as pd
import requests
from shapely.geometry import LineString, Point, Polygon

from emiproc.utils.osm import osm_json_to_gdf


logging.basicConfig(level=logging.INFO)

## Download Corvo from Openstreetmap

We will download everything available from Openstreetmap for the island of Corvo.
We will then see if we can extract some useful information from it.

Note: If you would do that for a larger place you might get a lot of data to process.
It could be a better idea to select only what you want.

This firt part of the code is a bit more technical but if you look at the result, it should be quite easy to understand.

In [ ]:

# Define the bounding box

# Corvo
# bbox = (39.5813, -31.3172, 39.8117, -30.7782)
# Faial
bbox = (38.5004, -28.8535, 38.6598, -28.5885)


In [ ]:


bbox = (39.5813, -31.3172, 39.8117, -30.7782)
url = "https://overpass-api.de/api/interpreter"
query = f"""[out:json];
nwr({bbox[0]},{bbox[1]},{bbox[2]},{bbox[3]});
out;"""
response = requests.get(url, params={"data": query})
data = response.json()

In [ ]:
data_by_id = {x["id"]: x for x in data["elements"]}

In [ ]:
data_by_id[13543350]

In [ ]:
data_by_id[1011532224]

In [ ]:
data_by_id[9332831444]

In [ ]:

gdf = osm_json_to_gdf(data)

gdf = gdf.set_index("id")


In [ ]:
gdf.iloc[::-1].explore(
    'name',
    # Add some transparency
    alpha=0.5,
)